# Text Study

After the initial audio study and the necessary text extraction, the next logical step will be the stduy of this type of unstructured data.

We will take a look at both the audio transcripts and the captions from the videos, in order to extract as much information as possible to predict the virality of a video.

## First libraries and variables import

In [12]:
import sys
from pathlib import Path
import os

# Get the absolute path of the folder containing the module
root_dir = Path.cwd().resolve().parent.parent

# Add the folder path to sys.path
sys.path.append(str(root_dir))

from nltk.tokenize import word_tokenize

In [2]:
from text_utils import load_json, load_transcriptions, clean_text, plot_freq_dist, analyze_text
from config.variables import text_path, json_file

In [3]:
text_folder = os.path.join(root_dir, text_path)
json_path = os.path.join(root_dir, json_file)

## Data recopilation and ordering

In [4]:
# Load the transcriptions of the videos
transcriptions = load_transcriptions(text_folder)

# Load the information from the JSON file
video_info = load_json(json_path)

In [5]:
# Create an object to store the combined information
video_data = {}

# Combine the information from transcriptions and the JSON file
for video_id, info in video_info.items():
    if video_id in transcriptions:
        general = info.get('general', '')  # Get the general information of the video or an empty string if not present
        text = general.get('text', '')  # Get the text of the video or an empty string if not present
        hashtags = general.get('hashtags', [])  # Get the hashtags of the video or an empty list if not present
        video_data[video_id] = {'transcription': transcriptions[video_id], 'text': text, 'hashtags': hashtags}

# Example of accessing the combined information for the first video
first_video = list(video_data.keys())[0]
print("Transcription:", video_data[first_video]['transcription'])
print("Text:", video_data[first_video]['text'])
print("Hashtags:", video_data[first_video]['hashtags'])

Transcription: 
Text: Confidence went 📈
Hashtags: []


In [6]:
# Counting the elements that are not empty strings in transcriptions
transcription_count = sum(1 for trans in transcriptions.values() if trans != '')
print("Number of non-empty transcriptions:", transcription_count)

# Counting the number of elements in video_data where the transcription is not ''
video_data_count_non_empty = sum(1 for video_id, data in video_data.items() if data['transcription'] != '')
print("Number of video_data elements with non-empty transcriptions:", video_data_count_non_empty)

# Counting the elements that are not empty strings in the video text
texts_count = sum(1 for video_id, data in video_data.items() if data['text'] != '')
print("Number of video_data elements with non-empty text:", texts_count)

# Counting the elements that are not empty lists in hashtags
hashtags_count = sum(1 for video_id, data in video_data.items() if data['hashtags'] != [])
print("Number of video_data elements with non-empty hashtags:", hashtags_count)


Number of non-empty transcriptions: 261
Number of video_data elements with non-empty transcriptions: 261
Number of video_data elements with non-empty text: 962
Number of video_data elements with non-empty hashtags: 853


As we can see, we have a transcription for 1 out of 4 videos aprox. Nevertheless, we have text and hashtags for almost all of the videos, so a text strudy could be deployed.

## Text Pre-Processing

### Text cleaning

In [7]:
# text cleaning of transcriptions and text
for video_id, data in video_data.items():
    if 'transcription' in data:
        video_data[video_id]['clean_transcription'] = clean_text(data['transcription'])
    if 'text' in data:
        video_data[video_id]['clean_text'] = clean_text(data['text'])

In [8]:
video_data[list(video_data.keys())[34]]

{'transcription': "is there the top 10 strongest one piece characters by the end of the show at number 10 we have Trafalgar law he can make anyone Immortal at the expense of his life number 9 we have useless kid he challenged kaido even though the odds are against him just like Luffy number 8 we have Sabo even though he's not at his Peak he can challenge fujitora and number 7 we have GARP the only man known to go to Toe with the former pirate king Goldie Rodger number 6 we have a kind of the author himself Oda said that if a kind of was the main character he'd find the one piece in a single year number five we have Shanks at the marineford where he just came in and said war is over and it really ended what else do you need to know about Shanks make sure like for part two",
 'text': '#Top10 Strongest #onepiece Characters by the end of the show. #anime #strongestcharacters #animeboy #luffy #zoro #animeedit #animeedits #animestiktok',
 'hashtags': ['top10',
  'onepiece',
  'anime',
  'str

As we can see, it has occured a tokenization, lemmatization and an elimination of hashtags, punctuation signs, special characters and stopwords for both text and transcription, bearing in mind the language of the possible text and transcription.

### Exploratory analysis

The next step will be the frequency distribution and the word cloud of the clean transcripts and texts, as well as the hashtags (probably the most useful one).

In [13]:
# Collect clean transcripts, clean texts, and hashtags from video_info
clean_transcripts = []
clean_texts = []
hashtags = []
for info in video_data.values():
    if 'clean_transcription' in info:
        clean_transcripts.extend(word_tokenize(info['clean_transcription']))
    if 'clean_text' in info:
        clean_texts.extend(word_tokenize(info['clean_text']))
    if 'hashtags' in info:
        hashtags.extend(info['hashtags'])

In [17]:
# Plot frequency distribution and word cloud for clean transcripts
print("Analysis of Transcripts:")
plot_freq_dist(clean_transcripts)
analyze_text(' '.join(clean_transcripts))

Analysis of Transcripts:
<FreqDist with 2013 samples and 5252 outcomes>
Most common words: [('like', 84), ('know', 67), ('get', 46), ('going', 46), ('yeah', 46), ('want', 41), ('people', 40), ('one', 40), ('really', 35), ('way', 32)]


In [18]:
# Plot frequency distribution and word cloud for clean texts
print("Analysis of Captions:")
plot_freq_dist(clean_texts)
analyze_text(' '.join(clean_texts))

Analysis of Captions:
<FreqDist with 1955 samples and 2838 outcomes>
Most common words: [('reply', 59), ('one', 17), ('video', 15), ('day', 13), ('1', 12), ('know', 12), ('love', 11), ('jij', 10), ('antwoorden', 10), ('get', 10)]


In [19]:
# Plot frequency distribution and word cloud for hashtags
print("Analysis of Hashtags:")
plot_freq_dist(hashtags)
analyze_text(' '.join(hashtags))

Analysis of Hashtags:
<FreqDist with 2220 samples and 5330 outcomes>
Most common words: [('fyp', 417), ('foryou', 272), ('foryoupage', 174), ('fy', 116), ('fitness', 87), ('workout', 73), ('voorjou', 72), ('viral', 65), ('animeedit', 63), ('anime', 50)]
